<div style="font-size: 14pt;">Prof. Krzysztof Rybinski</div><br/><br/>
<div style="font-size: 22pt;"><b>Artificial Intelligence course</b></div><br/><br/>
<div style="font-size: 18pt;">LAB 6.2</div><br/>
<div style="font-size: 18pt;">- Programming CNN in Keras</div><br/><br/>

# MNIST dataset

In [1]:
import numpy as np
import keras
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

y_train = to_categorical(train_labels)
y_test  = to_categorical(test_labels)

# normalize inputs from 0-255 to 0.0-1.0
train_images = train_images.astype('float32')
test_images  = test_images.astype('float32')
train_images = train_images / 255.0
test_images  = test_images / 255.0

# training dataset
print('Training dataset:')
print(train_images.shape)
print(train_labels.shape)
print(y_train.shape)

# testing dataset
print('Testing dataset:')
print(test_images.shape)
print(test_labels.shape)
print(y_test.shape)

2023-03-22 11:06:22.879888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training dataset:
(60000, 28, 28)
(60000,)
(60000, 10)
Testing dataset:
(10000, 28, 28)
(10000,)
(10000, 10)


# Baseline - Multilayer perceptron (MLP)

In [2]:
train_images.shape

(60000, 28, 28)

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

#try different activation functions
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(300, activation='relu'))
model.add(Dense(10,  activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 10)                3010      
                                                                 
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________


2023-03-22 11:06:26.349662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [5]:
model.fit(train_images, y_train, validation_split=0.1, epochs=10, batch_size=32)

Epoch 1/10
1688/1688 [==============================] - 2s 980us/step - loss: 0.2329 - accuracy: 0.9332 - val_loss: 0.0990 - val_accuracy: 0.9715
Epoch 2/10
1688/1688 [==============================] - 2s 913us/step - loss: 0.0941 - accuracy: 0.9716 - val_loss: 0.0741 - val_accuracy: 0.9770
Epoch 3/10
1688/1688 [==============================] - 2s 913us/step - loss: 0.0617 - accuracy: 0.9809 - val_loss: 0.0816 - val_accuracy: 0.9765
Epoch 4/10
1688/1688 [==============================] - 2s 910us/step - loss: 0.0432 - accuracy: 0.9859 - val_loss: 0.0780 - val_accuracy: 0.9780
Epoch 5/10
1688/1688 [==============================] - 2s 922us/step - loss: 0.0313 - accuracy: 0.9903 - val_loss: 0.0730 - val_accuracy: 0.9807
Epoch 6/10
1688/1688 [==============================] - 2s 926us/step - loss: 0.0242 - accuracy: 0.9919 - val_loss: 0.0754 - val_accuracy: 0.9795
Epoch 7/10
1688/1688 [==============================] - 2s 932us/step - loss: 0.0203 - accuracy: 0.9935 - val_loss: 0.0709 -

In [6]:
model.evaluate(test_images, y_test)

313/313 [==============================] - 0s 566us/step - loss: 0.0928 - accuracy: 0.9763


[0.09275301545858383, 0.9763000011444092]

# Convolutional neural networks

## One convolution layer

In [7]:
from keras.layers import Conv2D, MaxPooling2D, Reshape, Activation, Dropout, Flatten
from keras.models import Sequential

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4725818188990629097
xla_global_id: -1
]


Check Keras documentation <br/>
- https://keras.io/api/layers/convolution_layers/convolution2d/
- https://keras.io/api/layers/pooling_layers/max_pooling2d/

In [9]:
#build computational graph
model = Sequential()

model.add(Reshape((28, 28, 1), input_shape=(28, 28)))
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# add Dense layers
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='softmax'))


print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 20)                62740     
                                                                 
 dense_3 (Dense)             (None, 10)                210       
                                                      

In [10]:
#compile
model.compile(loss='categorical_crossentropy', 
              optimizer="adam", metrics=['categorical_accuracy'])

In [11]:
#train
model.fit(train_images, y_train, validation_split=0.1, epochs=10, batch_size=32)

Epoch 1/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2457 - categorical_accuracy: 0.9268 - val_loss: 0.0840 - val_categorical_accuracy: 0.9773
Epoch 2/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0837 - categorical_accuracy: 0.9744 - val_loss: 0.0659 - val_categorical_accuracy: 0.9823
Epoch 3/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0595 - categorical_accuracy: 0.9818 - val_loss: 0.0572 - val_categorical_accuracy: 0.9865
Epoch 4/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0479 - categorical_accuracy: 0.9845 - val_loss: 0.0549 - val_categorical_accuracy: 0.9837
Epoch 5/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0373 - categorical_accuracy: 0.9884 - val_loss: 0.0519 - val_categorical_accuracy: 0.9860
Epoch 6/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0314 - categorical_accuracy: 0.9904 - val_loss: 0.0535 - val_categorical_accuracy:

In [12]:
#evaluate on test images
model.evaluate(test_images, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0491 - categorical_accuracy: 0.9869


[0.04908600449562073, 0.9868999719619751]

In [13]:
#calculate predicted labels
ypred = model.predict( test_images )
ypred_class = np.argmax( ypred, axis=1 )

313/313 [==============================] - 0s 1ms/step


In [14]:
#print confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels , ypred_class)

array([[ 970,    0,    3,    1,    1,    1,    2,    0,    2,    0],
       [   0, 1134,    0,    0,    0,    0,    0,    0,    1,    0],
       [   2,    6, 1010,    1,    1,    0,    0,    6,    5,    1],
       [   0,    0,    3,  998,    0,    4,    0,    2,    3,    0],
       [   0,    1,    0,    0,  971,    0,    2,    0,    1,    7],
       [   2,    1,    0,   10,    0,  870,    4,    0,    4,    1],
       [   5,    3,    0,    1,    1,    1,  945,    0,    2,    0],
       [   0,    2,    4,    1,    1,    0,    0, 1014,    2,    4],
       [   2,    0,    0,    1,    0,    1,    0,    2,  966,    2],
       [   0,    2,    0,    3,    5,    2,    0,    5,    1,  991]])

## Two convolution layers

### Example

Build the following model
<center><img src="img/cnn_exercise.png" width="100%"></center>

In [15]:
model = Sequential()
model.add(Reshape((28,28,1), input_shape=(28,28)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu'), )
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.2) )
model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='relu'), )
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.2) )
model.add(Flatten())
# add Dense layers
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2) )
model.add(Dense(10, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 3136)             

In [16]:
#compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer="adam", metrics=['categorical_accuracy'])

In [17]:
#free computer memory
import gc
gc.collect()

3313

In [18]:
model.fit(train_images, y_train, validation_split=0.1, epochs=10, batch_size=32)

Epoch 1/10
1688/1688 [==============================] - 30s 18ms/step - loss: 0.2710 - categorical_accuracy: 0.9152 - val_loss: 0.0553 - val_categorical_accuracy: 0.9858
Epoch 2/10
1688/1688 [==============================] - 31s 19ms/step - loss: 0.1090 - categorical_accuracy: 0.9675 - val_loss: 0.0421 - val_categorical_accuracy: 0.9883
Epoch 3/10
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0793 - categorical_accuracy: 0.9763 - val_loss: 0.0389 - val_categorical_accuracy: 0.9897
Epoch 4/10
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0640 - categorical_accuracy: 0.9803 - val_loss: 0.0348 - val_categorical_accuracy: 0.9903
Epoch 5/10
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0580 - categorical_accuracy: 0.9822 - val_loss: 0.0371 - val_categorical_accuracy: 0.9915
Epoch 6/10
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0468 - categorical_accuracy: 0.9855 - val_loss: 0.0345 - val_categoric

In [19]:
#evaluate on test images
model.evaluate(test_images, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.0268 - categorical_accuracy: 0.9927


[0.026790399104356766, 0.9926999807357788]